In [148]:
from pandas import read_csv
import numpy as np
from collections import Counter
from pprint import pprint
import math
import csv

In [150]:
dataFrame = read_csv('data/training.csv',header = None)
#print(dataFrame)

In [151]:
trainingLabels = dataFrame.iloc[:,-1].values
totalDocuments = len(trainingLabels)
print(trainingLabels)
print("the length of trainningLabels is ====  " + str(totalDocuments))

[14 16  8 ... 15 13 17]
the length of trainningLabels is ====  12000


In [152]:
dataFrame.drop(dataFrame.columns[[-1,0]],axis = 1, inplace=True)
#print(dataFrame)

In [153]:
trainingClassCount = Counter()
for val in trainingLabels:
    trainingClassCount[val] += 1
   
pprint(trainingClassCount)
print(len(trainingClassCount))

Counter({16: 651,
         9: 649,
         11: 646,
         4: 643,
         12: 639,
         15: 637,
         6: 630,
         10: 628,
         13: 626,
         2: 624,
         3: 622,
         14: 621,
         7: 618,
         8: 614,
         5: 602,
         18: 593,
         17: 580,
         1: 483,
         19: 467,
         20: 427})
20


In [154]:
allLabels = np.unique(trainingLabels)
print(allLabels)
totalClassCount = len(allLabels)
print(totalClassCount)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
20


In [155]:
sortedClassCount = sorted(trainingClassCount.items())
#print(sortedClassCount)

classCount = [y for x,y in sortedClassCount]
#print(classCount)
logPriorProb = [math.log(x/totalDocuments) for x in classCount]
#pprint(logPriorProb)

In [156]:
trainingMatrix = np.array(dataFrame.values)
totalWords = len(trainingMatrix[0])
#print(totalWords)
#print(trainingMatrix)
#print("the size of training matrix is:    "  + str(len(trainingMatrix)) + "  X  " +  str(len(trainingMatrix[0])))

In [157]:
dividedOnClass = [[x for x, y in zip(trainingMatrix,trainingLabels) if y == c] for c in allLabels]


#pprint(dividedOnClass)
#print(len(dividedOnClass))


In [158]:
tempTable = [np.array(val).sum(axis=0)for val in dividedOnClass]

#pprint (tempTable)

In [159]:
smoother = 1 / totalWords
#print(smoother)
smoothCount = np.array(tempTable) + smoother
#pprint(smoothCount)
#print(len(smoothCount))

In [160]:

    
totalWordInEachClass = [sum(val) for val in tempTable]
#pprint(totalWordInEachClass)

In [161]:
totalWordInEachClassSmooth = np.array(totalWordInEachClass) + 1
#print(totalWordInEachClassSmooth)


In [162]:
likelihood = np.array([ x / y for x , y in zip (smoothCount,totalWordInEachClassSmooth)])
#pprint(likelihood)
#print(len(likelihood))

In [163]:
logLikelihood = np.array([np.log(val) for val in likelihood])
#pprint(logLikelihood)
#print(len(logLikelihood))

In [164]:
#pprint(logPriorProb)
#pprint(logLikelihood)

In [165]:
testingDataFrame = read_csv('data/testing.csv', header = None)
testingId = testingDataFrame.iloc[:,0].values
#pprint(testingDataFrame)
#print(testingId)
testingDataFrame.drop(testingDataFrame.columns[0,],axis=1,inplace=True)


In [166]:


#print(testingDataFrame)


In [167]:
testingVals = np.array(testingDataFrame.values)
#print(testingVals)
probEstimation = [logLikelihood.dot(val) for val in testingVals]
#print(probEstimation)




In [168]:
totalProbEstimation = [[x + y for x,y in zip(logPriorProb, val)] for val in probEstimation]
#print(totalProbEstimation)

In [169]:
prediction = [val.index(max(val))+1 for val in totalProbEstimation]
#print(prediction)

In [180]:
csvData = [ [docId , p]  for docId,p in zip (testingId,prediction)]
pprint(csvData)


[[12001, 9],
 [12002, 8],
 [12003, 3],
 [12004, 16],
 [12005, 7],
 [12006, 16],
 [12007, 18],
 [12008, 1],
 [12009, 15],
 [12010, 5],
 [12011, 10],
 [12012, 15],
 [12013, 9],
 [12014, 14],
 [12015, 8],
 [12016, 1],
 [12017, 1],
 [12018, 4],
 [12019, 1],
 [12020, 15],
 [12021, 14],
 [12022, 18],
 [12023, 9],
 [12024, 9],
 [12025, 9],
 [12026, 15],
 [12027, 4],
 [12028, 15],
 [12029, 2],
 [12030, 5],
 [12031, 17],
 [12032, 6],
 [12033, 5],
 [12034, 9],
 [12035, 4],
 [12036, 6],
 [12037, 14],
 [12038, 3],
 [12039, 15],
 [12040, 4],
 [12041, 7],
 [12042, 17],
 [12043, 7],
 [12044, 14],
 [12045, 10],
 [12046, 13],
 [12047, 17],
 [12048, 4],
 [12049, 13],
 [12050, 15],
 [12051, 18],
 [12052, 15],
 [12053, 19],
 [12054, 10],
 [12055, 4],
 [12056, 13],
 [12057, 5],
 [12058, 18],
 [12059, 17],
 [12060, 9],
 [12061, 12],
 [12062, 2],
 [12063, 1],
 [12064, 14],
 [12065, 11],
 [12066, 3],
 [12067, 8],
 [12068, 14],
 [12069, 1],
 [12070, 5],
 [12071, 12],
 [12072, 11],
 [12073, 5],
 [12074, 19],
 [

 [14261, 6],
 [14262, 14],
 [14263, 16],
 [14264, 6],
 [14265, 15],
 [14266, 19],
 [14267, 6],
 [14268, 1],
 [14269, 10],
 [14270, 15],
 [14271, 15],
 [14272, 12],
 [14273, 11],
 [14274, 11],
 [14275, 2],
 [14276, 16],
 [14277, 11],
 [14278, 17],
 [14279, 7],
 [14280, 18],
 [14281, 13],
 [14282, 3],
 [14283, 18],
 [14284, 15],
 [14285, 13],
 [14286, 18],
 [14287, 5],
 [14288, 12],
 [14289, 9],
 [14290, 2],
 [14291, 10],
 [14292, 2],
 [14293, 6],
 [14294, 3],
 [14295, 5],
 [14296, 14],
 [14297, 7],
 [14298, 15],
 [14299, 5],
 [14300, 6],
 [14301, 3],
 [14302, 2],
 [14303, 18],
 [14304, 14],
 [14305, 5],
 [14306, 3],
 [14307, 8],
 [14308, 3],
 [14309, 8],
 [14310, 19],
 [14311, 20],
 [14312, 3],
 [14313, 3],
 [14314, 1],
 [14315, 5],
 [14316, 19],
 [14317, 7],
 [14318, 11],
 [14319, 11],
 [14320, 18],
 [14321, 10],
 [14322, 1],
 [14323, 13],
 [14324, 18],
 [14325, 3],
 [14326, 16],
 [14327, 14],
 [14328, 6],
 [14329, 16],
 [14330, 3],
 [14331, 12],
 [14332, 18],
 [14333, 17],
 [14334, 17

 [16261, 14],
 [16262, 4],
 [16263, 8],
 [16264, 15],
 [16265, 12],
 [16266, 4],
 [16267, 20],
 [16268, 15],
 [16269, 14],
 [16270, 19],
 [16271, 12],
 [16272, 7],
 [16273, 14],
 [16274, 6],
 [16275, 17],
 [16276, 9],
 [16277, 6],
 [16278, 14],
 [16279, 8],
 [16280, 2],
 [16281, 15],
 [16282, 6],
 [16283, 10],
 [16284, 3],
 [16285, 1],
 [16286, 14],
 [16287, 4],
 [16288, 13],
 [16289, 3],
 [16290, 5],
 [16291, 10],
 [16292, 1],
 [16293, 14],
 [16294, 6],
 [16295, 9],
 [16296, 16],
 [16297, 16],
 [16298, 4],
 [16299, 4],
 [16300, 16],
 [16301, 16],
 [16302, 12],
 [16303, 7],
 [16304, 9],
 [16305, 19],
 [16306, 2],
 [16307, 1],
 [16308, 12],
 [16309, 4],
 [16310, 19],
 [16311, 10],
 [16312, 15],
 [16313, 4],
 [16314, 5],
 [16315, 19],
 [16316, 9],
 [16317, 16],
 [16318, 7],
 [16319, 14],
 [16320, 18],
 [16321, 4],
 [16322, 12],
 [16323, 18],
 [16324, 2],
 [16325, 8],
 [16326, 3],
 [16327, 2],
 [16328, 2],
 [16329, 17],
 [16330, 17],
 [16331, 9],
 [16332, 8],
 [16333, 1],
 [16334, 17],
 [

 [18431, 10],
 [18432, 2],
 [18433, 8],
 [18434, 3],
 [18435, 20],
 [18436, 15],
 [18437, 9],
 [18438, 18],
 [18439, 11],
 [18440, 16],
 [18441, 3],
 [18442, 7],
 [18443, 14],
 [18444, 14],
 [18445, 12],
 [18446, 8],
 [18447, 7],
 [18448, 2],
 [18449, 2],
 [18450, 18],
 [18451, 15],
 [18452, 11],
 [18453, 1],
 [18454, 8],
 [18455, 7],
 [18456, 12],
 [18457, 8],
 [18458, 11],
 [18459, 8],
 [18460, 8],
 [18461, 15],
 [18462, 17],
 [18463, 13],
 [18464, 13],
 [18465, 20],
 [18466, 2],
 [18467, 8],
 [18468, 13],
 [18469, 15],
 [18470, 9],
 [18471, 16],
 [18472, 17],
 [18473, 17],
 [18474, 13],
 [18475, 6],
 [18476, 16],
 [18477, 3],
 [18478, 1],
 [18479, 15],
 [18480, 19],
 [18481, 19],
 [18482, 10],
 [18483, 14],
 [18484, 11],
 [18485, 3],
 [18486, 4],
 [18487, 14],
 [18488, 19],
 [18489, 2],
 [18490, 3],
 [18491, 14],
 [18492, 15],
 [18493, 18],
 [18494, 8],
 [18495, 18],
 [18496, 20],
 [18497, 17],
 [18498, 14],
 [18499, 19],
 [18500, 13],
 [18501, 19],
 [18502, 4],
 [18503, 20],
 [1850

In [181]:
csvData = [['id','class']] + csvData
with open('data/submission.csv', 'w',newline='') as csvFile:
    
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()